In [46]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
# !pip install --upgrade mxnet-cu100
# !pip install autogluon
# !pip install -U ipykernel

--2020-05-21 10:48:46--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2571184 (2.5M) [text/plain]
Saving to: ‘doctor_train.csv.1’

doctor_train.csv.1  100%[===================>]   2.45M  --.-KB/s    in 0.1s    

2020-05-21 10:48:47 (24.2 MB/s) - ‘doctor_train.csv.1’ saved [2571184/2571184]

--2020-05-21 10:48:49--  https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [0]:
# import autogluon as ag
# from autogluon import TabularPrediction as task
import time
import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
tdf = pd.read_csv('doctor_test.csv')

cols = ['age','Money','day','Time','Doctor_visits','last_visit',
        'cured_in','Profession','Status','edu','Irregular','residence',
        'prev_diagnosed','communication','Month','side_effects']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_y = le.fit(df['Y'])
df['Y'] = le_y.transform(df['Y'])

In [0]:
# df.info()
def dataessing(df):
  cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed','communication','Month','side_effects']
  num_col = ['age','day','Time','Doctor_visits','last_visit','cured_in']
  loan = ['Money']
  target = ['Y']
  dfc = pd.DataFrame()
  dfc[cat_col] = df[cat_col].fillna("unknown")
  dfc[num_col+loan] = df[num_col+loan].fillna(0)
  dfc[num_col] = dfc[num_col].clip(lower=0)
  drop_val = ['last_visit','cured_in']
  dfc.drop(drop_val,axis=1,inplace=True)
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(dfc, df['Y'], random_state=45, test_size=0.3)
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = dataessing(df)

In [50]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    cat_features=X_train.columns.value
    # iterations=10, depth=10, learning_rate=0.01, loss_function='Logloss', 
    verbose=True)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(y_test, y_hat))
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))

CatBoostError: ignored

In [0]:
def preprocessing(tr,te):
  lab = 'Y'
  # drop index
  tr.drop('ID',1,inplace=True)
  te.drop('ID',1,inplace=True)
  # impyte missing vales using mean
  tr = tr.fillna(0)
  te = te.fillna(0)
  # get_dummies
  tr_DUM = pd.get_dummies(tr.drop(lab,1))
  te_DUM = pd.get_dummies(te)
  # load new col name
  cols = tr_DUM.columns.values
  #split for validation
  from sklearn.model_selection import train_test_split
  X_train, X_test, y_train, y_test = train_test_split(tr_DUM, tr[lab], random_state=0, test_size=0.2)
  # SMOTE for training data only!
  from imblearn.over_sampling import SMOTE
  smt = SMOTE(ratio='auto')
  X_sm, y_sm = smt.fit_resample(X_train, y_train)
  # convert to dataframe and labels back
  X_sm = pd.DataFrame(X_sm)
  X_sm.columns = cols
  X_sm['Y'] = y_sm # train data prepared
  X_test['Y'] = y_test # validation data prepared
  return X_sm, X_test, te_DUM

train_DF, valid_DF, test_DF = preprocessing(df,tdf)

0    4939
1     701
Name: Y, dtype: int64

In [11]:
%%time
predictor = task.fit(train_data=train_DF, 
                     label='Y', 
                    #  eval_metric='roc_auc',
                     eval_metric='accuracy',
                     auto_stack=True,
                     verbosity=2)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200521_095004/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20200521_095004/
Train Data Rows:    39896
Train Data Columns: 55
Preprocessing data ...
Here are the first 10 unique label values in your data:  [0 1]
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Feature Generator processed 39896 data points with 54 features
Original Features:
	float features: 54
	object features: 0
Generated Features:
	int features: 0
All Features:
	float features: 54
	object features: 0
	int features: 0
	Data preprocessing and feature engineering runtime = 0.37s ...
AutoGluon will gauge predictive performance using evaluatio

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-03f7c4101bb8>", line 1, in <module>
    get_ipython().run_cell_magic('time', '', "predictor = task.fit(train_data=train_DF, \n                     label='Y', \n                    #  eval_metric='roc_auc',\n                     eval_metric='accuracy',\n                     auto_stack=True,\n                     verbosity=2)")
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2117, in run_cell_magic
    result = fn(magic_arg_s, cell)
  File "<decorator-gen-60>", line 2, in time
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/magics/execution.py", line 1193, in time
    exec(code, glob, local

KeyboardInterrupt: ignored

In [0]:
y_pred = predictor.predict(valid_DF.drop('Y',1))
perf = predictor.evaluate_predictions(valid_DF['Y'], y_pred)
results = predictor.fit_summary()

In [0]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(accuracy_score(valid_DF['Y'], y_pred))
print(confusion_matrix(valid_DF['Y'], y_pred))
print(classification_report(valid_DF['Y'], y_pred))

In [0]:
y_hat = predictor.predict(test_DF)

In [7]:
Y = le_y.inverse_transform(y_hat)
output = pd.DataFrame({'id': tdf.ID,
                       'Y': Y})
output.to_csv('submissionSMOTETomek.csv', index=False)
from google.colab import files
files.download('submissionSMOTETomek.csv')

NameError: ignored